In [60]:
# Import API modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import geopandas as gpd
# Import linestrings
from shapely.geometry import LineString

In [141]:
# Read token.txt
with open("inrix_token.txt", "r") as f:
    token = f.read()

# Use API
endpoint = f"https://api.iq.inrix.com/v1/segments/speed?box=37.757386%7C-122.490667%2C37.746138%7C-122.395481"
r = requests.get(endpoint, headers={'Authorization': f"Bearer {token}"})

In [ ]:
od = 'origin'
geoFilterType = 'polygon'
radius = '0.2km'
points = '37.734622%7C-122.471603%2C37.743627%7C-122.463850%2C37.743066%7C-122.475429'
providerType = 'consumer'
startDateTime = '%3E%3D2023-06-01T02%3A31'
endDateTime = '%3C%3D2023-06-15T02%3A31'

# Trade Areas API
endpoint_TA = f'https://api.iq.inrix.com/v1/trips?od={od}&geoFilterType={geoFilterType}&radius={radius}&points={points}&providerType={providerType}&startDateTime={startDateTime}&endDateTime={endDateTime}'
r_TA = requests.get(endpoint_TA, headers={'Authorization': f"Bearer {token}"})

In [87]:
# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(r_TA.json()['data'])


In [88]:
# Separate startLoc into two columns: all numbers before comma and all numbers after comma, then convert to floats
df['start_lat'] = df['startLoc'].apply(lambda x: x.split(',')[0]).astype(float)
df['start_lon'] = df['startLoc'].apply(lambda x: x.split(',')[1]).astype(float)
df['end_lat'] = df['endLoc'].apply(lambda x: x.split(',')[0]).astype(float)
df['end_lon'] = df['endLoc'].apply(lambda x: x.split(',')[1]).astype(float)

# Create linestring from start and end coordinates
df['geometry'] = df.apply(lambda x: LineString([(x['start_lon'], x['start_lat']), (x['end_lon'], x['end_lat'])]), axis=1)

In [89]:
# Read geojson
sf = gpd.read_file('San_Francisco_Bay_Region_2020_Census_Tracts.geojson')

In [103]:
# Convert df to geopandas
df = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# Find intersection of linestring and geojson
sf['intersections'] = sf.geometry.clip(df.geometry)

In [120]:
sf[sf['intersections'].notnull()].intersections

10      MULTILINESTRING ((-121.95581 37.41945, -121.95...
15      LINESTRING (-122.16838 37.42140, -122.15505 37...
18      MULTILINESTRING ((-122.15101 37.42561, -122.14...
31      LINESTRING (-121.99771 37.35218, -121.99621 37...
32      LINESTRING (-122.01503 37.36660, -122.01385 37...
                              ...                        
1760    MULTILINESTRING ((-122.23309 38.01152, -122.21...
1761    LINESTRING (-122.27059 38.03080, -122.24849 38...
1762    LINESTRING (-122.36987 37.88468, -122.36347 37...
1763    MULTILINESTRING ((-122.25037 37.99224, -122.24...
1764    MULTILINESTRING ((-122.28885 37.96306, -122.28...
Name: intersections, Length: 697, dtype: geometry

### Temporal Processing

In [124]:
df['startDateTime'] = pd.to_datetime(df['startDateTime'], format='%Y-%m-%dT%H:%M:%S.%fZ')
df['endDateTime'] = pd.to_datetime(df['endDateTime'], format='%Y-%m-%dT%H:%M:%S.%fZ')

df['duration'] = (df['endDateTime'] - df['startDateTime']).dt.total_seconds()

df['day'] = df['startDateTime'].dt.day
df['month'] = df['startDateTime'].dt.month
df['year'] = df['startDateTime'].dt.year
df['dow'] = df['startDateTime'].dt.dayofweek
df['start_hour'] = df['startDateTime'].dt.hour
df['end_hour'] = df['endDateTime'].dt.hour
df['start_date'] = df['startDateTime'].dt.date
df['end_date'] = df['endDateTime'].dt.date
df['weekend'] = df['dow'].apply(lambda x: 1 if x >= 5 else 0)

## Weather API?

In [131]:
with open("weather_token.txt", "r") as f:
    weather_token = f.read()

date = '2023-11-29'

city = 'San Francisco'
# Use API
endpoint_weather = f"http://api.weatherapi.com/v1/history.json?key={weather_token}&q={city}&dt={date}"
r_weather = requests.get(endpoint_weather)

r_weather.json()

{'location': {'name': 'San Francisco',
  'region': 'California',
  'country': 'United States of America',
  'lat': 37.78,
  'lon': -122.42,
  'tz_id': 'America/Los_Angeles',
  'localtime_epoch': 1701567162,
  'localtime': '2023-12-02 17:32'},
 'forecast': {'forecastday': [{'date': '2023-11-29',
    'date_epoch': 1701216000,
    'day': {'maxtemp_c': 13.2,
     'maxtemp_f': 55.7,
     'mintemp_c': 9.8,
     'mintemp_f': 49.6,
     'avgtemp_c': 11.3,
     'avgtemp_f': 52.3,
     'maxwind_mph': 8.5,
     'maxwind_kph': 13.7,
     'totalprecip_mm': 6.11,
     'totalprecip_in': 0.24,
     'avgvis_km': 8.7,
     'avgvis_miles': 5.0,
     'avghumidity': 75.0,
     'condition': {'text': 'Light rain shower',
      'icon': '//cdn.weatherapi.com/weather/64x64/day/353.png',
      'code': 1240},
     'uv': 3.0},
    'astro': {'sunrise': '07:04 AM',
     'sunset': '04:51 PM',
     'moonrise': '06:46 PM',
     'moonset': '09:40 AM',
     'moon_phase': 'Waning Gibbous',
     'moon_illumination': 97},
 

In [135]:
# Read as dataframe
df_weather = pd.DataFrame.from_dict(r_weather.json()['forecast']['forecastday'][0]['hour'])

In [136]:
df_weather

,time_epoch,time,temp_c,temp_f,is_day,condition,wind_mph,wind_kph,wind_degree,wind_dir,...,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv
0,1701244800,2023-11-29 00:00,12.1,53.8,0,"{'text': 'Patchy rain possible', 'icon': '//cd...",6.9,11.2,128,SE,...,41.8,0,0,0,0,10.0,6.0,11.6,18.7,1.0
1,1701248400,2023-11-29 01:00,12.0,53.6,0,"{'text': 'Patchy rain possible', 'icon': '//cd...",6.9,11.2,127,SE,...,43.8,0,0,0,0,10.0,6.0,11.1,17.9,1.0
2,1701252000,2023-11-29 02:00,11.7,53.1,0,"{'text': 'Overcast', 'icon': '//cdn.weatherapi...",6.7,10.8,117,ESE,...,51.1,0,0,0,0,10.0,6.0,10.5,17.0,1.0
3,1701255600,2023-11-29 03:00,11.9,53.4,0,"{'text': 'Patchy rain possible', 'icon': '//cd...",6.3,10.1,106,ESE,...,46.2,0,0,0,0,10.0,6.0,9.8,15.7,1.0
4,1701259200,2023-11-29 04:00,11.7,53.1,0,"{'text': 'Light rain', 'icon': '//cdn.weathera...",4.7,7.6,88,E,...,45.8,0,0,0,0,9.0,5.0,7.3,11.7,1.0
5,1701262800,2023-11-29 05:00,11.3,52.3,0,"{'text': 'Mist', 'icon': '//cdn.weatherapi.com...",4.9,7.9,53,NE,...,51.2,0,0,0,0,7.0,4.0,7.1,11.4,1.0
6,1701266400,2023-11-29 06:00,10.5,50.8,0,"{'text': 'Moderate rain', 'icon': '//cdn.weath...",8.1,13.0,82,E,...,46.7,0,0,0,0,7.0,4.0,11.6,18.7,1.0
7,1701270000,2023-11-29 07:00,10.5,50.8,0,"{'text': 'Light drizzle', 'icon': '//cdn.weath...",8.5,13.7,71,ENE,...,46.0,0,0,0,0,2.0,1.0,12.2,19.7,1.0
8,1701273600,2023-11-29 08:00,11.4,52.5,1,"{'text': 'Thundery outbreaks possible', 'icon'...",8.1,13.0,70,ENE,...,49.7,0,0,0,0,2.0,1.0,11.5,18.6,3.0
9,1701277200,2023-11-29 09:00,10.6,51.1,1,"{'text': 'Light drizzle', 'icon': '//cdn.weath...",8.3,13.3,70,ENE,...,44.6,0,0,0,0,2.0,1.0,11.1,17.9,3.0


In [147]:
import tradeAreasPreprocessing as tap

# Initialize class
tap = tap.tradeAreasPreprocessing(token=token,
                                  od = 'origin',
geoFilterType = 'polygon',
radius = '0.2km',
points = '37.734622%7C-122.471603%2C37.743627%7C-122.463850%2C37.743066%7C-122.475429',
providerType = 'consumer',
startDateTime = '%3E%3D2023-06-01T02%3A31',
endDateTime = '%3C%3D2023-06-15T02%3A31')

# Read data
df = tap.read_data()

In [151]:
df = tap.create_dataframe(df['data'])

In [152]:
df = tap.separate_coordinates(df)